# Perplexity playbook

<p>
Mal Minhas, v0.7<br>
25.01.25
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.3</b>: 15.01.24. First version</li>
<li><b>v0.4</b>: 02.06.24. Updated for latest Perplexity API</li>
<li><b>v0.5</b>: 01.08.24. Updated again for latest Perplexity API and llama 3.1 support</li>
<li><b>v0.6</b>: 09.11.24. Updated to include support for retrieving citations</li>
<li><b>v0.7</b>: 25.01.25. Updated to latest Perplexity models and added streaming support</li>
</ul>
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an OpenAI-compliant API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api` per the documetnation [here](https://docs.perplexity.ai/).  Perplexity regularly update the model support for this API and at the time of writing the latest and greatest supported model is `llama-3.1`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/guides/model-cards):
```
Model	Context Length	Model Type
sonar-pro	200k	Chat Completion
sonar	127k	Chat Completion
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import display, Markdown

def renderTextAsMarkdown(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get('PERPLEXITY_API_KEY')
DEFAULT_MODEL = 'sonar'
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = 'Be precise and concise.'
MID_LEVEL_DETAIL = 'Explain in a few sentences.'
GREAT_DETAIL = 'Explain in a lot of detail.'
POLITE_DETAIL = 'You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user.'
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return r.json()

def renderResponse(d):
    return renderTextAsMarkdown(d.get("choices")[0].get("message").get("content"))

def getModel(d):
    return renderTextAsMarkdown(d.get("model"))

def renderCitations(d):
    def formatReferences(url_list):
        return [f"[{i + 1}]: {url}" for i, url in enumerate(url_list)] 
    # Print citations in a Markdown-compatible format
    markdown_output = "\n".join(formatReferences(d.get("citations", [])))
    print(markdown_output)

Let's test the function on an example query using the default `llama-3.1-sonar-small-128k-online` model and then `llama-3.1-8b-instruct`.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
d = getPerplexityResponse(prompt)
renderResponse(d)

To get your washing machine repaired in Southampton, you should contact a domestic appliance repair service. Trades such as appliance repair engineers or technicians are the professionals you need to call. Companies like NAC, Go Assist, and Domestec offer same or next-day repair services for washing machines in Southampton, covering various brands and models[1][2][3]. You can book their services online or by phone to arrange a visit from an experienced engineer.

CPU times: user 9.72 ms, sys: 3.61 ms, total: 13.3 ms
Wall time: 3.32 s


As of 09.11.24, Perplexity are offering citations as part of the API response.  We can pull them out as follows.  The URLs all work and point to valid companies that could help with this job:

In [4]:
renderCitations(d)

[1]: https://nacrepair.co.uk/appliance-repairs/washing-machine-repair/washing-machine-repair-southampton/
[2]: https://go-assist.co.uk/appliance-repairs/washing-machine-repairs/washing-machine-repair-southampton
[3]: https://domestec.co.uk/southampton-appliance-repairs/
[4]: https://www.southampton-appliance-repairs.co.uk/help/terms-conditions
[5]: https://www.harrapdomesticappliances.co.uk/p/appliances-repairs-and-spares


In [5]:
%%time

renderResponse(getPerplexityResponse(prompt, model='sonar'))

To get your washing machine repaired in Southampton, you should contact a domestic appliance repair service. Trades such as appliance repair engineers or technicians are the professionals you need to call. Companies like NAC, Go Assist, and Domestec offer same-day or next-day repair services for washing machines in Southampton, covering various brands and models[1][2][3]. You can book their services online or by phone to have your washing machine diagnosed and fixed promptly.

CPU times: user 13.6 ms, sys: 7.36 ms, total: 21 ms
Wall time: 3.58 s


In [6]:
d = getPerplexityResponse(prompt, model='sonar-pro', direction=POLITE_DETAIL)
renderResponse(d)
renderCitations(d)

If you have a broken washing machine in Southampton, UK, you have several options to get it repaired. Here's what you can do:

1. Call an appliance repair service: There are several reputable appliance repair companies in Southampton that specialize in washing machine repairs. Some options include:

- NAC Repair: They offer same-day or next-day washing machine repairs in Southampton[1][7].
- Go Assist: They provide local washing machine repair services with experienced engineers[3].
- Harrap Domestic Appliances: A local company offering washing machine repairs in Southampton[4].

2. Contact a local technician: You can search for independent appliance repair technicians in Southampton who may offer more flexible scheduling and competitive pricing.

3. Check with the manufacturer: If your washing machine is still under warranty, contact the manufacturer directly as they may offer free repairs or send an authorized technician.

The trade you need to call is typically referred to as an "appliance repair technician" or "domestic appliance engineer." These professionals are trained to diagnose and fix issues with various household appliances, including washing machines.

When contacting a repair service, be prepared to provide:

- The make and model of your washing machine
- A description of the problem you're experiencing
- Your location in Southampton
- Your availability for a repair appointment

Many repair services offer online booking or phone consultations to make the process easier. They often provide upfront pricing and may be able to complete the repair on the first visit if they carry common spare parts with them[1][3].

Remember to choose a reputable service with good reviews and proper qualifications to ensure a quality repair for your washing machine.

[1]: https://nacrepair.co.uk/appliance-repairs/washing-machine-repair/washing-machine-repair-southampton/
[2]: https://appliancerepairsouthampton.ny-biz.co
[3]: https://go-assist.co.uk/appliance-repairs/washing-machine-repairs/washing-machine-repair-southampton
[4]: https://www.harrapdomesticappliances.co.uk/p/appliances-repairs-and-spares
[5]: https://www.southamptonnyappliancerepair.com
[6]: https://www.southampton-appliance-repairs.co.uk/help/terms-conditions
[7]: https://nacrepair.co.uk/appliance-repair/appliance-repair-southampton/
[8]: https://speedyrefrigeratorservice.com/southampton/
[9]: https://appliancecare-southampton.co.uk/terms-conditions/
[10]: https://www.aaqualitydomesticappliances.co.uk


In [7]:
%%time

renderResponse(getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=POLITE_DETAIL))

If you have a broken washing machine in Southampton, UK, there are several options for getting it repaired. Here are a few reliable services you can consider:

1. **NAC Repair**: NAC offers same-day or next-day washing machine repairs in Southampton and surrounding areas. They have a network of engineers who can diagnose and fix various types of washing machines, including front-loading, top-loading, semi-automatic, and fully automatic models[1].

2. **Raynor Appliance Service**: Raynor Appliance Service has over 40 years of experience in appliance repairs, including washing machines. They provide a one-year guarantee for parts and labor and serve the Southampton area with competitive pricing[2].

3. **Go Assist**: Go Assist offers trusted washing machine repair services in and around Southampton. They provide same or next-day service, competitive pricing, and a repair guarantee. Their engineers are experienced in repairing all makes and models of washing machines[3].

4. **Harrap Domestic Appliances**: Harrap offers comprehensive and affordable washing machine repair services in Southampton. Their highly skilled team can diagnose and fix various issues with your washing machine, and they also provide advice on upgrading if necessary[4].

To get your washing machine mended, you should call one of these appliance repair services. Here’s how you can proceed:

- **NAC Repair**: Call them or book online quickly and easily.
- **Raynor Appliance Service**: Contact them via phone or email.
- **Go Assist**: Call them on 0333 733 1234 or book online.
- **Harrap Domestic Appliances**: Ring them on 02380771585.

Each service has its own unique benefits, so it might be helpful to compare their services and guarantees before making a decision.

CPU times: user 12.2 ms, sys: 7.26 ms, total: 19.5 ms
Wall time: 3.95 s


Let's see how it handles a query with a more detailed response:

In [8]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
renderResponse(getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=GREAT_DETAIL))

To fix a broken garden gate that is no longer closing properly, you have several options depending on your level of DIY expertise and the extent of the damage. Here are the steps you can take:

### DIY Approach

1. **Evaluate the Gate**:
   - Check if the gate is sagging or if the hinges are loose. This can often be fixed by tightening loose screws or replacing damaged hinges [2][4].

2. **Inspect Hinges and Latches**:
   - Look for rust, wear, or missing screws on the hinges and latches. Tighten any loose screws or replace them if necessary [4].

3. **Adjust Gate Alignment**:
   - Use a level to ensure the gate is even. If it's sagging, you might need to realign it using a turnbuckle kit or by adjusting the hinges [2][4].

4. **Replace Hardware**:
   - If the hinges are damaged, purchase new ones. Opt for heavy-duty hinges if your gate is large or heavy [4].

5. **Reinforce Posts**:
   - If the posts supporting the gate are wobbly or leaning, use concrete mix and post levelers to reinforce them [4].

### Professional Help

1. **Call a Fence and Gate Repair Service**:
   - If the damage is extensive or you're not comfortable with DIY repairs, consider hiring a professional fence and gate repair service. They can assess the gate and provide a solution that ensures it functions properly and securely [1][5].

2. **Check Local Reviews**:
   - Look for local professionals with good reviews on platforms like Angi to ensure you're hiring someone reliable and skilled [5].

### Steps for Professional Repair

1. **Assessment**:
   - The professional will evaluate the gate's condition, including the hinges, latches, and posts.
   
2. **Repair or Replacement**:
   - Depending on the extent of the damage, they may need to replace hinges, latches, or even parts of the gate frame.
   
3. **Reinstallation**:
   - Once the necessary repairs are made, they will reinstall the gate ensuring it swings freely and closes securely.

By following these steps, whether you choose to DIY or hire a professional, you can ensure your garden gate is fixed and functioning properly again.

CPU times: user 8.84 ms, sys: 7.66 ms, total: 16.5 ms
Wall time: 3.67 s


## 3. Streaming interface

Leveraging the OpenAI interface:

In [9]:
import asyncio
from openai import OpenAI
from typing import List

citations = {}
references = []

async def getPerplexityResponseStreaming(question:str, references:List, direction:str=MID_LEVEL_DETAIL) -> str:
    messages = [
        {
            "role": "system",
            "content": (direction),
        },
        {   
            "role": "user",
            "content": (question),
        },
    ]
    client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")
    # chat completion with streaming
    response_stream = client.chat.completions.create(
        model="sonar-pro",
        messages=messages,
        stream=True,
    )
    for response in response_stream:
        if response.citations:
            references += response.citations
        yield response.choices[0].delta.content

async def processStreamingResponse(prompt, references):
    async for ref in getPerplexityResponseStreaming(prompt, references):
        print(ref, end="")  # Print each result as soon as it's yielded without adding newlines

# Jupyter-specific way to run asyncio in a cell
prompt = "How many stars are there in our galaxy?"

# Use an event loop in Jupyter to run the async function
await processStreamingResponse(prompt, references)
print(f"\n------------ References ------------")
citations['citations'] = references
renderCitations(citations)

The Milky Way galaxy contains an estimated 100 to 400 billion stars[1][3][6]. This wide range reflects the difficulty in precisely counting stars in our galaxy. Astronomers estimate this number by calculating the galaxy's mass and the percentage of that mass made up of stars[6]. The exact figure depends on factors like the average mass of stars used in calculations and our ability to detect very faint, low-mass stars throughout the galaxy[3][6]. While we can't count each star individually, these estimates give us a sense of the vast scale of our home galaxy.
------------ References ------------
[1]: https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/
[2]: https://www.littlepassports.com/blog/space/how-many-stars-are-in-the-universe/
[3]: https://www.space.com/25959-how-many-stars-are-in-the-milky-way.html
[4]: https://www.youtube.com/watch?v=Py2nZYmvTKg
[5]: https://www.space.com/26078-how-many-stars-are-there.html
[6]: https://asd.gsfc.nasa.gov/blueshif